In [ ]:
# 포지셔널 인코딩 클래스
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=100):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.attention_weights = nn.Parameter(torch.randn(input_size, 1))

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

# 시계열 트랜스포머 모델 클래스
class TimeSeriesTransformerModel(nn.Module):
    def __init__(self, num_features, d_model, n_heads, num_encoder_layers, d_ff, dropout_rate, lstm_hidden_size, num_lstm_layers, target_size):
        super(TimeSeriesTransformerModel, self).__init__()
        self.d_model = d_model

        # LSTM Layer
        self.lstm = nn.LSTM(input_size=num_features, hidden_size=lstm_hidden_size, num_layers=num_lstm_layers, batch_first=True)
        
        # Linear layer to transform LSTM output to match Transformer d_model size
        self.linear1 = nn.Linear(lstm_hidden_size, d_model)

        # Positional Encoding
        self.pos_encoder = PositionalEncoding(d_model, dropout_rate)
        
        # Transformer Encoder
        encoder_layers = nn.TransformerEncoderLayer(d_model, n_heads, d_ff, dropout_rate)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_encoder_layers)
        
        # Output layer
        self.out = nn.Linear(d_model, target_size)

    def forward(self, src):
        # LSTM layer
        lstm_out, _ = self.lstm(src)
        # lstm_out: [batch_size, sequence_length, lstm_hidden_size]

        # Transform LSTM output to match Transformer d_model size
        src = self.linear1(lstm_out) * math.sqrt(self.d_model)
        # src: [batch_size, sequence_length, d_model]

        # Positional Encoding and Transformer
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        # output: [batch_size, sequence_length, d_model]

        # 가중치 평균 계산을 위한 어텐션 적용
        attention_weights = torch.softmax(self.attention_weights, dim=0)
        output = torch.matmul(output.transpose(1, 2), attention_weights).squeeze(-1)
        # output: [batch_size, d_model]

        # 선형 레이어를 통과시켜 최종 출력 생성
        output = torch.sigmoid(self.out(output))
        # output: [batch_size, target_size]

        return output

LSTM과 트랜스포머의 강점
LSTM:

시계열 데이터에서 장기적인 의존성과 순차적 정보를 포착하는 데 뛰어납니다.
데이터 포인트의 순서가 중요할 때 특히 효과적입니다.
트랜스포머:

시퀀스를 병렬로 처리하는 데 효율적이며, 이는 LSTM의 순차적 처리보다 빠를 수 있습니다.
자기 주의 메커니즘을 통해 모델이 입력 시퀀스의 다른 부분에 집중할 수 있게 하여, 시퀀스 내 먼 요소 간의 관계를 식별하는 데 능숙합니다.
LSTM과 트랜스포머 결합하기
LSTM으로 순차적 처리:

시계열 데이터를 순차적으로 처리하기 위해 LSTM 레이어로 시작할 수 있습니다. 이 단계는 모델이 데이터에 내재된 시간적 관계를 포착할 수 있게 합니다.
트랜스포머로 병렬 처리:

LSTM의 출력을 트랜스포머 모듈에 공급할 수 있습니다. 트랜스포머는 자기 주의 메커니즘을 활용하여 시퀀스를 더 분석하고, LSTM이 놓칠 수 있는 복잡하고 장거리 의존성을 포착할 수 있습니다.
하이브리드 아키텍처:

최종 아키텍처는 하나 이상의 LSTM 레이어에 이어 하나 이상의 트랜스포머 블록을 가질 수 있습니다. 트랜스포머의 출력은 최종 예측이나 분류를 위한 전방 피드-포워드 네트워크를 통과할 수 있습니다.